##### Začneme tím, že na-import-ujeme knihovny:

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

##### Začneme stahování od roku 2018 a postupně budeme přidávat starší soubory.

In [2]:
response = requests.get('http://www.sukl.cz/2012')

In [3]:
soup = BeautifulSoup(response.text, "lxml")

Odkazy v HTML se začínají písmenkem a mají atribut "href".  Zároveň při prvním pohledu na stránku http://www.sukl.cz/2018 vidíme, že názvy složek, které potřebujeme se začínají slovem "Distribuce":

In [4]:
links = soup.find_all('a')

In [5]:
for slovo in links:
    
    if slovo.text.startswith('DistribuceLP_'):
        x ='http://www.sukl.cz' + slovo.attrs['href'] #Po několika pokusů jsme zjistily, že potřebujeme přidat 'http://www.sukl.cz'
        print(x) #Vytiskneme každý odkaz, abychom mohly prověřit, jestli opravdu fungují

http://www.sukl.cz/file/74315_1_1
http://www.sukl.cz/file/74314_1_1
http://www.sukl.cz/file/74313_1_1
http://www.sukl.cz/file/74312_1_1
http://www.sukl.cz/file/74311_1_1
http://www.sukl.cz/file/74310_1_1
http://www.sukl.cz/file/74309_1_1
http://www.sukl.cz/file/74308_1_1
http://www.sukl.cz/file/74307_1_1
http://www.sukl.cz/file/74306_1_1
http://www.sukl.cz/file/74305_1_1
http://www.sukl.cz/file/76216_1_1


##### Vytvoříme seznam a nahrajeme do toho naší odkazy:

In [ ]:
our_list = ['']
i = 0
for slovo in links:
    if slovo.text.startswith('DistribuceLP_'):
        x ='http://www.sukl.cz' + slovo.attrs['href']
        if i == 0:
            our_list[i] = x
        else:
            our_list.append(x)
        i+=1
print(our_list)

['http://www.sukl.cz/file/74315_1_1', 'http://www.sukl.cz/file/74314_1_1', 'http://www.sukl.cz/file/74313_1_1', 'http://www.sukl.cz/file/74312_1_1', 'http://www.sukl.cz/file/74311_1_1', 'http://www.sukl.cz/file/74310_1_1', 'http://www.sukl.cz/file/74309_1_1', 'http://www.sukl.cz/file/74308_1_1', 'http://www.sukl.cz/file/74307_1_1', 'http://www.sukl.cz/file/74306_1_1', 'http://www.sukl.cz/file/74305_1_1', 'http://www.sukl.cz/file/76216_1_1']


##### Vytvoříme seznam "our_files", do kterého nahrajeme každý ze souboru s pomocí seznamu odkazů:

In [ ]:
our_files = [ pd.read_excel(x, encoding='utf-8') for x in our_list ]

In [ ]:
our_filesX = our_files

##### Abychom mohly prověřit, jestli všechno je správně, vypišeme počet řádků v každém že souborů a pak součet:

In [ ]:
number_of_rows =[0]*12 #budeme používat seznam obsahující 12 čísel (protože 12 měsíců)

for i in range(0, len(our_files)):
    number_of_rows[i] = our_files[i].shape[0]
number_of_rows

In [ ]:
sum(number_of_rows) 

### Vsimly jsme si, ze tabulky pro rok 2011 a 2012 nemaji prvni sloupec "Obdobi" a proto pridame ten sloupec ke kazde z tabulek. Abychom zachovaly stejné netradiční formát datumu, vyplnily ten sloupec jako “201x.MM”, kde x značí rok 1 (2011) nebo 2 (2012) a MM značí měsíc. 

Abychom zachovaly stejné netradiční formát datumu, přidaly jsme do tabulek sloupec a vyplnily ho jako “201x.MM”, kde x značí rok 2011 nebo 2012 a MM značí měsíc. Pro vyplnění období uvnitř cyklu jsme použily metodu “.format”, která automaticky doplňovala číslo měsíců. (Pozn.: Chtěly jsme prvně spojit tabulky a následně až řešit problém špatného formátu.)

In [ ]:
for i in range(0, len(our_files)):
    our_files[i].insert(loc=0, column='Období', value='2012.{:02d}'.format(i+1))

##### Teď konečně spojíme složky. Víme, že mají stejnou strukturu (počet a názvy sloupců), proto nahrajeme soubory jeden za druhým. Budeme postupovat podle vzoru:

year_201x = pd.concat([our_files[0], our_files[1], "a tak dale" ], ignore_index = True), kde x je 2012 anebo 2012

In [ ]:
for i in range(1, len(our_files)):
    our_files[i] = pd.concat([our_files[i-1], our_files[i]], ignore_index = True)
year_2012 = our_files[i]
year_2012.head()

Zkontrolujeme, jestli počet řádků je stejný, jak v "sum(number_of_rows)", což je 72154 pro rok 2018

In [ ]:
year_2012.shape

In [ ]:
year_2012.tail()

##### Hura! Ted muzeme exportovat tuto slozku:

In [ ]:
year_2012.to_csv('years/year_2012.csv', index = False, encoding='utf-8')

###### Pro rok 2011 pouzijeme stejny postup!